## Double-loop framework implementation with Fossil Case (using integrated ultra-supercritical power plant model)


This notebook presents an implementation of the double-loop framework for the fossil case study using the integrated ultra-supercritical power plant model with a molten-salt based thermal energy storage.

First, import all necessary modules and packages.

In [1]:
# Import Python library packages
from types import ModuleType
import pandas as pd
from importlib import resources

# Import Pyomo packages
import pyomo.environ as pyo

# Import IDAES packages
from idaes.apps.grid_integration import Tracker
from idaes.apps.grid_integration import Bidder
from idaes.apps.grid_integration import PlaceHolderForecaster
from idaes.apps.grid_integration import DoubleLoopCoordinator
from idaes.apps.grid_integration.model_data import GeneratorModelData

# Import Prescient simulator
from prescient.simulator import Prescient

# Import integrated ultra-supercritical power plant with energy storage model
from multiperiod_double_loop_usc import MultiPeriodUsc


Interactive Python mode detected; using default matplotlib backend for plotting.


Set up fossil generator characteristics using a dictionary and double-loop model parameters. The fossil generator is the integrated ultra-supercritical power plant. The double-loop model parameters are the number of hours for the bidding and tracking models, number of scenarios to use, and the number of days for simulation.

In [2]:
generator_data = {
    "gen_name": "102_STEAM_3",
    "generator_type": "thermal",
    "p_min": 286,
    "p_max": 460,
    "min_down_time": 4,
    "min_up_time": 8,
    "ramp_up_60min": 60,
    "ramp_down_60min": 60,
    "shutdown_capacity": 286,
    "startup_capacity": 286,
    "production_cost_bid_pairs": [
        (286, 22.16602),
        (350, 19.0723),
        (400, 18.29703),
        (430, 17.71727),
        (460, 17.71727),
    ],
    "startup_cost_pairs": [(4, 7355.42), (10, 10488.35), (12, 11383.41)],
    "fixed_commitment": 1,
}
model_data = GeneratorModelData(**generator_data)

tracking_horizon = 10  # hours
bidding_horizon = 48  # hours
n_scenario = 1  # for bidding
n_tracking_hour = 1  # advance n_tracking_hour (i.e. assume we solve every hour)
num_days = 2


Set up the solver and the forecaster objects.

In [3]:
# create solver
solver = pyo.SolverFactory("ipopt")
solver.options = {
    "max_iter": 200,
}

# create forecaster object
with resources.open_text("idaes.apps.grid_integration.examples", "lmp_forecasts_concat.csv") as f:
    price_forecasts_df = pd.read_csv(f)
forecaster = PlaceHolderForecaster(price_forecasts_df=price_forecasts_df)



Set up the objects for Tracker, Bidder, and Coordinator.

In [4]:
# Tracker
mp_usc_tracker = MultiPeriodUsc(
    horizon=tracking_horizon,
    model_data=model_data
)

thermal_tracker = Tracker(
    tracking_model_object=mp_usc_tracker,
    n_tracking_hour=n_tracking_hour,
    solver=solver,
)

# Projection Tracker
mp_usc_projection_tracker = MultiPeriodUsc(
    horizon=tracking_horizon,
    model_data=model_data
)

thermal_projection_tracker = Tracker(
    tracking_model_object=mp_usc_projection_tracker,
    n_tracking_hour=n_tracking_hour,
    solver=solver,
)

# Bidder
mp_usc_bidder = MultiPeriodUsc(
    horizon=bidding_horizon,
    model_data=model_data
)
thermal_bidder = Bidder(
    bidding_model_object=mp_usc_bidder,
    n_scenario=n_scenario,
    solver=solver,
    forecaster=forecaster,
)

# Coordinator
coordinator = DoubleLoopCoordinator(
    bidder=thermal_bidder,
    tracker=thermal_tracker,
    projection_tracker=thermal_projection_tracker,
)


>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time

Set up the plugin module needed for the Prescient simulator

In [5]:
class PrescientPluginModule(ModuleType):
    def __init__(self, get_configuration, register_plugins):
        self.get_configuration = get_configuration
        self.register_plugins = register_plugins


plugin_module = PrescientPluginModule(
    get_configuration=coordinator.get_configuration,
    register_plugins=coordinator.register_plugins,
)


Get all generators data for Prescient simulator

In [6]:
rts_gmlc_data_dir = "C:\\grid\\source_code\\Prescient\\downloads\\rts_gmlc\\RTS-GMLC\\RTS_Data\\SourceData"


Declare the options for Prescient simulator

In [7]:
options = {
    "data_path": rts_gmlc_data_dir,
    "input_format": "rts-gmlc",
    "simulate_out_of_sample": True,
    "run_sced_with_persistent_forecast_errors": True,
    "output_directory": "bidding_plugin_test_multiperiod_rankine",
    "start_date": "2020-07-10",
    "num_days": num_days,
    "sced_horizon": tracking_horizon,
    "ruc_horizon": bidding_horizon,
    "compute_market_settlements": True,
    "day_ahead_pricing": "LMP",
    "ruc_mipgap": 0.01,
    "symbolic_solver_labels": True,
    "reserve_factor": 0.0,
    "deterministic_ruc_solver": "gurobi",
    "output_ruc_solutions": True,
    "sced_solver": "gurobi",
    "print_sced": True,
    "enforce_sced_shutdown_ramprate": True,
    "plugin": {
        "doubleloop": {
            "module": plugin_module,
            "bidding_generator": "102_STEAM_3",
        }
    },
}


Run Prescient simulator.

In [ ]:
Prescient().simulate(**options)


Initializing simulation...


C:\Users\susarlan\Anaconda3\envs\gmlc\lib\site-packages\egret\parsers\rts_gmlc\parser.py:256: FutureWarning: 
        Use pd.to_datetime instead.

  df = pd.read_csv(file_name,
C:\Users\susarlan\Anaconda3\envs\gmlc\lib\site-packages\egret\parsers\rts_gmlc\parser.py:256: FutureWarning: 
        Use pd.to_datetime instead.

  df = pd.read_csv(file_name,
C:\Users\susarlan\Anaconda3\envs\gmlc\lib\site-packages\egret\parsers\rts_gmlc\parser.py:256: FutureWarning: 
        Use pd.to_datetime instead.

  df = pd.read_csv(file_name,
C:\Users\susarlan\Anaconda3\envs\gmlc\lib\site-packages\egret\parsers\rts_gmlc\parser.py:256: FutureWarning: 
        Use pd.to_datetime instead.

  df = pd.read_csv(file_name,
C:\Users\susarlan\Anaconda3\envs\gmlc\lib\site-packages\egret\parsers\rts_gmlc\parser.py:304: FutureWarning: 
        Use pd.to_datetime instead.

  df = pd.read_csv(file_name,
C:\Users\susarlan\Anaconda3\envs\gmlc\lib\site-packages\egret\parsers\rts_gmlc\parser.py:304: FutureWarning: 
     

Setting default t0 state in RTS-GMLC parser


Dates to simulate: 2020-07-10 to 2020-07-11
RUC activation hours: 0
Final RUC date: 2020-07-11
Using current day's forecasts for RUC solves
Using persistent forecast error model when projecting demand and renewables in SCED


Extracting scenario to simulate
Ipopt 3.13.2: max_iter=200


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientif

INFO:egret:Calculating PTDF Matrix Factorization


...

  23  6.3851319e+08 1.28e+06 1.76e+02  -1.0 5.56e+07    -  1.99e-01 1.29e-01f  1
  24  6.3843035e+08 1.27e+06 1.74e+02  -1.0 6.21e+07    -  1.12e-01 1.07e-02f  1
  25  6.3729198e+08 2.93e+06 1.51e+02  -1.0 6.97e+07    -  3.66e-01 1.37e-01f  1
  26  6.3489410e+08 1.06e+07 1.19e+02  -1.0 9.87e+07    -  1.85e-01 2.06e-01f  1
  27  6.3467053e+08 1.05e+07 1.17e+02  -1.0 9.06e+07    -  5.03e-01 2.09e-02f  1
  28  6.3400278e+08 1.20e+07 1.06e+02  -1.0 1.08e+08    -  5.56e-01 9.75e-02f  1
  29  6.2953920e+08 4.64e+07 1.77e+01  -1.0 8.65e+07    -  7.10e-01 8.33e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  30  6.2935213e+08 2.37e+07 1.05e+01  -1.0 5.06e+07    -  3.00e-01 4.91e-01h  1
  31  6.2935693e+08 2.27e+07 1.57e+04  -1.0 8.43e+05  -4.0 7.30e-03 3.91e-02h  1
  32  6.2935698e+08 2.27e+07 1.57e+04  -1.0 8.09e+05  -4.5 8.81e-02 4.13e-04h  1
  33  6.2935768e+08 2.26e+07 6.72e+04  -1.0 7.89e+05  -5.0 3.88e-02 4.34e-03h  1
  34  6.2952870e+08 1.18e+06

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  6.3694047e+08 3.20e+03 9.19e+01  -1.0 9.10e+06    -  1.51e-01 1.91e-01f  1
  11  6.3217724e+08 4.27e+03 3.05e+01  -1.0 1.07e+07    -  6.84e-01 3.36e-01f  1
  12  6.3209791e+08 4.26e+03 3.55e+01  -1.0 3.19e+07    -  1.80e-01 2.48e-03f  1
  13  6.3108526e+08 4.69e+03 5.91e+01  -1.0 3.58e+07    -  5.47e-01 4.04e-02f  1
  14  6.3097214e+08 4.44e+03 1.01e+02  -1.0 9.69e+06    -  9.86e-01 5.49e-02h  1
  15  6.2164752e+08 3.42e+05 8.77e+01  -1.0 2.93e+08    -  2.15e-02 1.27e-01f  1
  16  6.2048401e+08 3.44e+05 8.65e+01  -1.0 3.50e+08    -  2.73e-02 1.35e-02f  1
  17  6.2046243e+08 3.43e+05 8.65e+01  -1.0 1.78e+08    -  7.14e-03 7.21e-04h  1
  18  6.1630017e+08 8.49e+05 7.75e+01  -1.0 4.34e+08    -  2.37e-03 1.02e-01f  1
  19  6.1622250e+08 8.48e+05 7.79e+01  -1.0 3.27e+08    -  2.91e-01 1.35e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20  6.1318954e+08 7.75e+05

   1  6.0205759e+08 9.97e+04 9.93e+01  -1.0 1.45e+05    -  7.39e-03 3.07e-03h  1
   2  6.0208709e+08 9.89e+04 9.85e+01  -1.0 1.25e+05    -  3.83e-03 8.24e-03h  1
   3  6.0219177e+08 9.76e+04 9.74e+01  -1.0 9.83e+04    -  1.20e-02 1.29e-02h  1
   4  6.0275714e+08 9.19e+04 9.80e+01  -1.0 9.97e+04    -  2.98e-02 5.83e-02h  1
   5  6.0391456e+08 8.09e+04 8.06e+01  -1.0 9.14e+04    -  4.62e-01 1.19e-01h  1
   6  6.1186854e+08 2.40e+03 7.52e+01  -1.0 1.42e+05    -  2.67e-01 9.70e-01h  1
   7  6.1183573e+08 1.95e+03 6.44e+01  -1.0 1.78e+05    -  9.89e-01 1.90e-01h  1
   8  6.1071324e+08 1.87e+03 6.35e+01  -1.0 1.33e+07    -  2.35e-02 4.20e-02f  1
   9  6.1057261e+08 1.86e+03 4.76e+01  -1.0 1.67e+07    -  1.81e-01 5.99e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  6.0336608e+08 7.91e+03 4.86e+01  -1.0 2.02e+07    -  7.24e-02 2.59e-01f  1
  11  6.0328524e+08 7.89e+03 6.68e+01  -1.0 2.15e+07    -  8.19e-01 3.01e-03f  1
  12  6.0222881e+08 8.37e+03

   1  5.7416807e+08 2.00e+05 9.93e+01  -1.0 2.01e+05    -  1.11e-02 3.56e-03h  1
   2  5.7416938e+08 1.98e+05 9.84e+01  -1.0 2.00e+05    -  9.91e-03 9.64e-03h  1
   3  5.7424813e+08 1.95e+05 9.65e+01  -1.0 1.98e+05    -  2.49e-02 1.95e-02h  1
   4  5.7478326e+08 1.75e+05 8.92e+01  -1.0 1.95e+05    -  7.48e-02 1.02e-01h  1
   5  5.7613843e+08 1.28e+05 7.16e+01  -1.0 1.75e+05    -  2.32e-01 2.69e-01h  1
   6  5.7829277e+08 4.90e+04 2.43e+01  -1.0 1.28e+05    -  8.49e-01 6.17e-01h  1
   7  5.7847481e+08 1.31e+04 6.48e+00  -1.0 6.54e+05    -  9.36e-01 7.34e-01h  1
   8  5.7828387e+08 1.28e+04 7.43e+00  -1.0 9.41e+06    -  1.04e-01 1.66e-02h  1
   9  5.7635127e+08 9.46e+03 8.75e+00  -1.0 1.11e+07    -  3.02e-01 2.63e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  5.7477516e+08 8.05e+03 1.55e+01  -1.0 1.57e+07    -  2.94e-01 1.49e-01f  1
  11  5.7475883e+08 8.03e+03 3.24e+01  -1.0 8.97e+06    -  3.78e-01 2.63e-03h  1
  12  5.7418064e+08 3.80e+03

   1  5.4804169e+08 6.53e+05 7.76e+01  -1.0 6.57e+05    -  8.81e-01 5.99e-03h  1
   2  5.4805115e+08 6.01e+05 5.18e+01  -1.0 5.68e+06    -  4.18e-02 7.95e-02h  1
   3  5.4810225e+08 5.94e+05 5.09e+01  -1.0 3.79e+07    -  4.42e-03 1.16e-02h  1
   4  5.4817462e+08 5.85e+05 5.01e+01  -1.0 4.00e+07    -  2.42e-02 1.51e-02h  1
   5  5.4838050e+08 5.61e+05 4.80e+01  -1.0 3.94e+07    -  2.97e-02 4.26e-02h  1
   6  5.5192822e+08 5.46e+06 2.93e+03  -1.0 3.78e+07    -  3.08e-03 4.19e-01h  1
   7  5.5204873e+08 5.33e+06 2.86e+03  -1.0 2.20e+07    -  3.38e-01 2.43e-02h  1
   8  5.5442406e+08 5.63e+06 1.17e+03  -1.0 2.13e+07    -  5.79e-02 4.98e-01h  1
   9  5.5446201e+08 5.55e+06 1.16e+03  -1.0 1.04e+07    -  3.10e-01 1.41e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  5.5489596e+08 4.91e+06 1.01e+03  -1.0 9.60e+06    -  1.89e-01 1.26e-01h  1
  11  5.5519553e+08 4.54e+06 9.31e+02  -1.0 7.05e+06    -  1.45e-01 7.63e-02h  1
  12  5.5747849e+08 2.24e+06